In [1]:
import torch
import kornia
import numpy as np
import sys
sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
import cv2 as cv
from torch import autograd

In [2]:
f = 692.653256; cx = 629.321381; cy = 330.685425
p = np.array([ [692.653256, 0.000000, 629.321381, 0.000000],
[0.000000, 692.653256, 330.685425, 0.000000],
[0.000000, 0.000000, 1.000000, 0.000000] ])

In [3]:
def convert_pix_cam_coord(pix):
    px = (pix[0] - cx)/f
    py = (pix[1] - cy)/f
    return np.array([px,py,1])

In [4]:
def project_lid_on_img(lid_pt,R,t,p):
    lid_pt = np.append(lid_pt,[1])
    t = np.append(t,[1])
    T = np.append(R,np.array([0,0,0]).reshape(1,3),axis=0)
    T = np.append(T,t.reshape(4,1),axis=1)
    tran_pt =  np.dot(T,lid_pt)
    proj_lid_pt = np.dot(p,tran_pt).reshape(3,1)
    pix = np.array([proj_lid_pt[0]/proj_lid_pt[2],proj_lid_pt[1]/proj_lid_pt[2]]).reshape(2,1)
    return pix

In [5]:
def cost_function_rot(N, q, D):
    R = kornia.quaternion_to_rotation_matrix(q)
    D_rotated = torch.matmul(R,D.double())
    B = torch.mm(N,D_rotated)
    B = B**2
    return B

In [6]:
def cost_function_trans(N, q, T, pt_m):
    R = kornia.quaternion_to_rotation_matrix(q)
    m_rotated = torch.matmul(R.double(), pt_m.double())
    m_trans = torch.add(m_rotated, T)
    B = torch.mm(N,m_trans)
    B = B**2
    return B

In [7]:
cam_pnt_a = np.array([507,218])
cam_pnt_b = np.array([381,339])
cam_pnt_c = np.array([524,492])

N_ab = np.cross(convert_pix_cam_coord(cam_pnt_a), convert_pix_cam_coord(cam_pnt_b))
N_ab = N_ab/(np.sqrt(np.dot(N_ab,N_ab)))
N_bc = np.cross(convert_pix_cam_coord(cam_pnt_b), convert_pix_cam_coord(cam_pnt_c))
N_bc = N_bc/(np.sqrt(np.dot(N_bc,N_bc)))

Ncap = np.append(N_ab.reshape(3,1),N_bc.reshape(3,1),axis=1)
print(N_ab,N_bc)

[-0.67357658 -0.70141032 -0.23306255] [-0.7053016   0.65920346 -0.26076895]


In [8]:
lid_pnt_a = torch.FloatTensor([-0.475146,-0.195425,1.86017])
lid_pnt_b = torch.FloatTensor([-0.817135,0.122386,1.85952])
lid_pnt_c = torch.FloatTensor([-0.457918,0.543083,1.83382])

# R_temp = torch.FloatTensor([[0.866,0.5,0],[-0.5,0.866,0],[0,0,1]])

# lid_pnt_a = torch.mm(R_temp,lid_pnt_a.view(3,1))
# lid_pnt_b = torch.mm(R_temp,lid_pnt_b.view(3,1))
# lid_pnt_c = torch.mm(R_temp,lid_pnt_c.view(3,1))

dir_ab = lid_pnt_a - lid_pnt_b
dir_cb = lid_pnt_c - lid_pnt_b

dir = np.append(dir_ab.reshape(3,1), dir_cb.reshape(3,1), axis=1)
print(dir)

[[ 0.34198898  0.359217  ]
 [-0.317811    0.420697  ]
 [ 0.00065005 -0.02569997]]


In [9]:
q_init = np.array([0,0,0,1])
T_init = np.array([0,0,0])
q = autograd.Variable(torch.from_numpy(q_init.astype(dtype=float)),requires_grad=True)
T = autograd.Variable(torch.from_numpy(T_init.astype(dtype=float)),requires_grad=True)
D = autograd.Variable(dir_cb.view(3,1))
N = autograd.Variable(torch.from_numpy(N_bc.astype(dtype=float)))

In [10]:
total = autograd.Variable(torch.FloatTensor([0]))

In [11]:
for i in range(1000):
    total = cost_function_rot(N.view(1,3), q, D)
    total.backward()
    q.data -= .1 * q.grad.data
    q.data = kornia.normalize_quaternion(q.data)
print(total,q.data)

tensor([[1.0378e-09]], dtype=torch.float64, grad_fn=<PowBackward0>) tensor([ 0.0047, -0.0056, -0.0268,  0.9996], dtype=torch.float64)


In [12]:
R_final = kornia.quaternion_to_rotation_matrix(q.data).numpy()
print(R_final)

[[ 0.99850357  0.05347128 -0.01146517]
 [-0.05357574  0.99852318 -0.00900583]
 [ 0.01096668  0.00960661  0.99989372]]


In [13]:
total_trans = autograd.Variable(torch.FloatTensor([0]))

In [14]:
img = cv.imread("/home/chinmay/Downloads/test_image.png")

In [23]:
pt_m = (lid_pnt_c+lid_pnt_b)/2
Nbc = autograd.Variable(torch.from_numpy(N_bc.astype(dtype=float)))

# pt_m = (lid_pnt_b)
# Nbc = autograd.Variable(torch.from_numpy(N_bc.astype(dtype=float)))

# dir_ab_proj = project_lid_on_img(dir_ab, R, T)

for i in range(100):
    total_trans = cost_function_trans(Nbc.view(1,3), q, T.view(3,1), pt_m.view(3,1))
    total_trans.backward()
    T.data -= .000001 * T.grad.data
print(total_trans,T.data,T.grad.data)

pix_a = project_lid_on_img(lid_pnt_a,R_final,T.data.numpy(),p)
pix_b = project_lid_on_img(lid_pnt_b,R_final,T.data.numpy(),p)
pix_c = project_lid_on_img(lid_pnt_c,R_final,T.data.numpy(),p)
# pix_a = project_lid_on_img(lid_pnt_a,np.eye(3),np.zeros(3),p)
# pix_b = project_lid_on_img(lid_pnt_b,np.eye(3),np.zeros(3),p)
# pix_c = project_lid_on_img(lid_pnt_c,np.eye(3),np.zeros(3),p)

print(pix_a,pix_b,pix_c)

cv.circle(img, tuple(pix_a), 4, (255,0,0), thickness=2, lineType=8, shift=0);
cv.circle(img, tuple(pix_b), 4, (255,0,0), thickness=2, lineType=8, shift=0);
cv.circle(img, tuple(pix_c), 4, (255,0,0), thickness=2, lineType=8, shift=0);

cv.imshow('Transformed lidar pnts',img)
cv.waitKey(0)
cv.destroyAllWindows()

tensor([[0.0003]], dtype=torch.float64, grad_fn=<PowBackward0>) tensor([ 0.1538, -0.1437,  0.0569], dtype=torch.float64) tensor([-74.8467,  69.9547, -27.6728], dtype=torch.float64)
[[501.49307771]
 [210.94005426]] [[383.63376034]
 [332.68739181]] [[521.0838231 ]
 [479.63618002]]


In [ ]:
pix_a = project_lid_on_img(lid_pnt_a,R_final,T.data.numpy(),p)
pix_b = project_lid_on_img(lid_pnt_b,R_final,T.data.numpy(),p)
pix_c = project_lid_on_img(lid_pnt_c,R_final,T.data.numpy(),p)
# pix_a = project_lid_on_img(lid_pnt_a,np.eye(3),np.zeros(3),p)
# pix_b = project_lid_on_img(lid_pnt_b,np.eye(3),np.zeros(3),p)
# pix_c = project_lid_on_img(lid_pnt_c,np.eye(3),np.zeros(3),p)

print(pix_a,pix_b,pix_c)

In [ ]:
img = cv.imread("/home/chinmay/Downloads/test_image.png")
cv.circle(img, tuple(pix_a), 4, (255,0,0), thickness=2, lineType=8, shift=0);
cv.circle(img, tuple(pix_b), 4, (255,0,0), thickness=2, lineType=8, shift=0);
cv.circle(img, tuple(pix_c), 4, (255,0,0), thickness=2, lineType=8, shift=0);
# cv.imwrite("/home/chinmay/Desktop/output_30_rot.png",img)
cv.imshow('Transformed lidar pnts',img)
cv.waitKey(0)
cv.destroyAllWindows()